In [19]:
import glob, os
import tensorflow as tf
from itertools import groupby
from collections import defaultdict

# 因为我装的是CPU版本的，运行起来会有'warning'，解决方法入下，眼不见为净~  
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 数据源：http://vision.stanford.edu/aditya86/ImageNetDogs/
image_filenames = glob.glob( './imagenet-dogs/n02*/*.jpg' )

training_dataset = defaultdict(list)
testing_dataset = defaultdict(list)

image_filename_with_breed = list( map(lambda filename:(filename.split('/')[2], filename), image_filenames) )

for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x: x[0]):
    for i, breed_image in enumerate(breed_images):
        if i % 5 == 0:
            testing_dataset[dog_breed].append(breed_image[1])
        else:
            training_dataset[dog_breed].append(breed_image[1])

    breed_training_count = len(training_dataset[dog_breed])
    breed_testing_count = len(testing_dataset[dog_breed])
    
    assert round(breed_testing_count / (breed_training_count + breed_testing_count), 2) > 0.18, \
    "Not enough testing images."
    
    

def imageset_to_records_files(dataset, record_location, resize, \
                              channels = 1, image_type = 'jpg', one_record_count = 100):
    writer = None
    current_index = 0
    image_number = 0
    sess = tf.Session()
    
    for breed, images_filenames in dataset.items():
        for image_filename in images_filenames:
            if image_number % one_record_count == 0:
                if writer:
                    writer.close()
                    
                record_filename = "{record_location}-{current_index}.tfrecords". \
                format(record_location=record_location, current_index=current_index)

                writer = tf.python_io.TFRecordWriter(record_filename)
                current_index += 1
                    
            image_number += 1

            image_file = tf.read_file(image_filename)

            try:
                if( 'jpg' == image_type ):
                    image = tf.image.decode_jpeg(image_file, channels)
                elif( 'png' == image_type ):
                    image = tf.image.decode_png(image_file, channels)
            except:
                print('image decode fail: ', image_filename)
                continue

            resized_image = tf.image.resize_images( image, resize )

            #image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()
            image_loaded = sess.run(tf.cast(resized_image, tf.uint8))
            image_bytes = image_loaded.tobytes()
            image_label = breed.encode("utf-8")

            feature = {
                'label':tf.train.Feature(bytes_list=tf.train.BytesList(value = [image_label])),
                'image':tf.train.Feature(bytes_list=tf.train.BytesList(value = [image_bytes]))
            }
            
            example = tf.train.Example( features=tf.train.Features(feature = feature) )

            writer.write(example.SerializeToString())

    writer.close()                

# pil_imageset_to_records_files 的速度是 imageset_to_records_file 的好几倍
def pil_imageset_to_records_files(dataset, record_location, resize, \
                              channels = 1, image_type = 'jpg', one_record_count = 100):
    from PIL import Image
    writer = None
    current_index = 0
    image_number = 0
    
    for breed, images_filenames in dataset.items():
        for image_filename in images_filenames:
            if image_number % one_record_count == 0:
                if writer:
                    writer.close()
                    
                record_filename = "{record_location}-{current_index}.tfrecords". \
                format(record_location=record_location, current_index=current_index)

                writer = tf.python_io.TFRecordWriter(record_filename)
                current_index += 1
                    
            image_number += 1

            img = None
            try:
                img = Image.open( image_filename )
                if( 1 == channels ):
                    img = img.convert('L')
                img = img.resize( resize )
            
            except:
                print('image decode fail: ', image_filename)
                continue

            image_bytes = img.tobytes()
            image_label = breed.encode("utf-8")

            feature = {
                'label':tf.train.Feature(bytes_list=tf.train.BytesList(value = [image_label])),
                'image':tf.train.Feature(bytes_list=tf.train.BytesList(value = [image_bytes]))
            }
            
            example = tf.train.Example( features=tf.train.Features(feature = feature) )

            writer.write(example.SerializeToString())

    writer.close()


imageset_to_records_files(testing_dataset, './record_files/test-image/testing-image', [250, 151])
pil_imageset_to_records_files(training_dataset, './record_files/train-image/training-image', [250, 151])

print('convert finish.')



convert finish.


In [1]:
import tensorflow as tf
import glob

labels = list(map(lambda c: c.split("/")[-1], blob.glob("./imagenet-dogs/*")))
train_labels = tf.map_fn(lambda l: tf.where(tf.equal(labels, l))[0, 0:1][0], label_batch, dtype=tf.int64)



sess = tf.Session()

# records_path: './record_files/train-image/*.tfrecords'
# image_shape: [250, 151, 1]
# batch_size: 3
def read_images_from_tfrecords(records_path, image_shape, batch_size):
    file_path = tf.train.match_filenames_once( records_path )
    file_queue = tf.train.string_input_producer( file_path )
    reader = tf.TFRecordReader()
    _, serialized = reader.read( file_queue )

    feature = {
        'label':tf.FixedLenFeature([], tf.string),
        'image':tf.FixedLenFeature([], tf.string)
    }

    features = tf.parse_single_example( serialized, features = feature )

    record_image = tf.decode_raw(features['image'], tf.uint8)
    label = tf.cast( features['label'], tf.string )

    image = tf.reshape(record_image, [250, 151, 1])

    min_after_dequeue = 10
    capacity = min_after_dequeue + 3 * batch_size
    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size = batch_size, capacity = capacity, min_after_dequeue = min_after_dequeue)
    return image_batch, label_batch
    

batch_size = 3
image_batch, label_batch = read_images_from_tfrecords('./record_files/train-image/*.tfrecords', [250, 151, 1], batch_size)

# 初始化  
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())  
sess.run(init_op) 
coord = tf.train.Coordinator() 
threads = tf.train.start_queue_runners( sess = sess, coord = coord ) 

# print('file_path run: ', sess.run(file_path))
# print('record_image shape:', record_image.get_shape())
# print('record_image run: ', sess.run(record_image))

# print(image, label)
# print(sess.run(image), sess.run(label))
print(image_batch, label_batch)
print(sess.run(image_batch), sess.run(label_batch))


float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)
print('float_image_batch shape:', float_image_batch.get_shape())
print('float_image_batch: ', sess.run(float_image_batch))



#'''
conv2d_layer_one = tf.contrib.layers.convolution2d(
    float_image_batch, 
    num_outputs = 32, 
    kernel_size = (5, 5), 
    activation_fn = tf.nn.relu,
    weights_initializer = tf.random_normal_initializer,
    stride = (2, 2),
    data_format = 'NHWC',
    trainable = True
)

pool_layer_one = tf.nn.max_pool(conv2d_layer_one,
                               ksize = [1, 2, 2, 1],
                               strides = [1, 2, 2, 1],
                               padding = 'SAME')

print( conv2d_layer_one.get_shape() )
print( pool_layer_one.get_shape() )
#'''

#'''
conv2d_layer_two = tf.contrib.layers.convolution2d(
    pool_layer_one, 
    num_outputs = 64, 
    kernel_size = (5, 5), 
    activation_fn = tf.nn.relu,
    weights_initializer = tf.random_normal_initializer,
    stride = (1, 1),
    data_format = 'NHWC',
    trainable = True
)

pool_layer_two = tf.nn.max_pool(conv2d_layer_two,
                               ksize = [1, 2, 2, 1],
                               strides = [1, 2, 2, 1],
                               padding = 'SAME')

print( conv2d_layer_two.get_shape() )
print( pool_layer_two.get_shape() )
#'''


flattened_layer_two = tf.reshape( pool_layer_tow, [batch_size, -1] )
print( flattened_layer_two.get_shape() )

hidden_layer_three = tf.contrib.layers.fully_connected(
    flattened_layer_two, 
    512,
    weight_init = lambda i, dtype: tf.truncated_normal( [38912, 512], stddev = 0.1 ),
    activation_fn = tf.nn.relu
)

hidden_layer_three = tf.nn.dropout( hidden_layer_three, 0.1 )
final_fully_connected = tf.contrib.layers.fully_connected(
    hidden_layer_three,
    120,
    weight_init = lambda i, dtype: tf.truncated_normal( [512, 120], stddev = 0.1 )
)





#关闭线程  
coord.request_stop()  
coord.join(threads)  
sess.close()  

Tensor("shuffle_batch:0", shape=(3, 250, 151, 1), dtype=uint8) Tensor("shuffle_batch:1", shape=(3,), dtype=string)
[[[[134]
   [134]
   [134]
   ..., 
   [199]
   [200]
   [200]]

  [[199]
   [199]
   [200]
   ..., 
   [172]
   [174]
   [166]]

  [[172]
   [184]
   [203]
   ..., 
   [100]
   [101]
   [102]]

  ..., 
  [[ 14]
   [ 21]
   [ 14]
   ..., 
   [ 75]
   [ 82]
   [ 79]]

  [[ 72]
   [ 61]
   [ 65]
   ..., 
   [119]
   [ 64]
   [ 65]]

  [[ 40]
   [  0]
   [ 19]
   ..., 
   [ 39]
   [ 44]
   [ 53]]]


 [[[ 36]
   [ 36]
   [ 36]
   ..., 
   [ 50]
   [ 49]
   [ 47]]

  [[ 47]
   [ 47]
   [ 47]
   ..., 
   [ 36]
   [ 31]
   [ 31]]

  [[ 31]
   [ 34]
   [ 35]
   ..., 
   [ 49]
   [ 47]
   [ 46]]

  ..., 
  [[146]
   [161]
   [156]
   ..., 
   [162]
   [159]
   [153]]

  [[161]
   [176]
   [143]
   ..., 
   [108]
   [141]
   [162]]

  [[160]
   [169]
   [156]
   ..., 
   [125]
   [118]
   [ 91]]]


 [[[210]
   [210]
   [210]
   ..., 
   [191]
   [187]
   [187]]

  [[188]
   [188]
  

In [4]:
tf.contrib.layers.convolution2d?